In [37]:
import boto3
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the datafile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [38]:
#dropping observations w/ missing values
heart=heart.dropna()

In [39]:
#defining the input and target
x=heart[['male', 'age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'sysBP', 'glucose']]
y=heart[['TenYearCHD']]

#splitting the data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .2)

In [40]:
#scale the data
scaler = MinMaxScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train),columns = x_train.columns)
x_test =  pd.DataFrame(scaler.fit_transform(x_test),columns = x_test.columns)

In [41]:
#identifying important variables
lasso_md = LassoCV().fit(x_train,y_train)

#extract best lambda
lasso_lambda = lasso_md.alpha_

#fit LASSO with best lambda
lasso_md = Lasso(alpha=lasso_lambda).fit(x_train,y_train)
lasso_md.coef_

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 0.0650737 ,  0.25178849,  0.04356493,  0.        ,  0.        ,
       -0.        ,  0.40093954,  0.05763727])

In [42]:
#defining the input and target for logistic
x_train_logit= x_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
x_test_logit = x_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

#building logistic regression model
logit_md =LogisticRegression().fit(x_train_logit,y_train)

#predict on test
logit_pred = logit_md.predict_proba(x_test_logit)[:, 1]

#changing likelihoods to labels
logit_label = np.where(logit_pred < 0.1, 0, 1)

#computing the recall
recall_score(y_test, logit_label)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.808695652173913